This code generates lagged versions of specified columns (columns_to_process) for each geographic area (GEO_ID), ensuring proper handling of edge cases like missing prior-year data for 2010 and avoiding data leakage for 2019. The processed data, including the newly created lagged columns, is saved to a new CSV file, with warnings for any missing columns in the dataset.

In [ ]:
import pandas as pd

def process_lag_and_fill_columns(input_file_path, output_file_path, columns_to_process):
    """
    Creates lagged columns for a predefined list of columns.
    - Ensures 2010 lagged values are NaN (no prior data).
    - Fills 2019 lagged values with 2018 lagged values to avoid data leakage.

    Parameters:
    - input_file_path (str): Path to the input CSV file.
    - output_file_path (str): Path to save the updated CSV file.
    - columns_to_process (list of str): List of column names to lag.

    Returns:
    - None
    """
    # Load the main dataset
    df = pd.read_csv(input_file_path)

    # Sort the data by GEO_ID and Year to ensure proper order
    df = df.sort_values(by=['GEO_ID', 'Year'])
    print("Data sorted by GEO_ID and Year.")

    # Loop through each column in the list
    for col in columns_to_process:
        if col not in df.columns:
            print(f"Warning: Column '{col}' not found in the dataset. Skipping...")
            continue
        
        # Create lagged column
        lag_col_name = f"lag_{col}"
        df[lag_col_name] = df.groupby('GEO_ID')[col].shift(1)

        # For 2010, explicitly backfill lagged values with the same 2010 value
        df.loc[df['Year'] == 2010, lag_col_name] = df.loc[df['Year'] == 2010, col]

    # Save the updated DataFrame after processing all columns
    df.to_csv(output_file_path, index=False)
    print(f"Updated data with lagged columns saved to: {output_file_path}")

# Example usage
input_file_path = "/Users/avakrocheski-meyer/Downloads/TASK: 1119_start_to_finish/5_target_feature_engineering/target_features.csv"
output_file_path = "/Users/avakrocheski-meyer/Downloads/TASK: 1119_start_to_finish/6_feature_generation/lagged_columns.csv"
columns_to_process = ['TOT_HH', 'CIV_POP_18+', 'TOT_CIV_POP', 'CIV_POP_16', 'TOT_HOUSING_UNITS', 'TOT_POPULATION']

process_lag_and_fill_columns(input_file_path, output_file_path, columns_to_process)


Data sorted by GEO_ID and Year.
Updated data with lagged columns saved to: /u/ak5005/Python/TASK: 1119_start_to_finish/6_feature_generation/lagged_columns.csv


This code generates lagged variables for features specified in a separate file, ensuring that data is correctly grouped and sorted by GEO_ID and Year. It handles edge cases by backfilling 2010 values and saving the updated dataset with lagged variables to a new CSV file, while providing feedback on successful or incomplete interpolations for 2010.

In [ ]:
import pandas as pd

# File paths
features_file_path = "/Users/avakrocheski-meyer/Downloads/TASK: 1119_start_to_finish/3_feature_selection/final_features.csv"
data_file_path = "/Users/avakrocheski-meyer/Downloads/TASK: 1119_start_to_finish/6_feature_generation/lagged_columns.csv"
output_file_path = "/Users/avakrocheski-meyer/Downloads/TASK: 1119_start_to_finish/6_feature_generation/all_lagged.csv"

# Load the features from the file
features_df = pd.read_csv(features_file_path)
features_to_lag = features_df['Features'].tolist()  # Adjust column name if necessary

# Load the data
df = pd.read_csv(data_file_path)

# Ensure the data is sorted by GEO_ID and Year for proper lagging
df = df.sort_values(by=['GEO_ID', 'Year'])

# Create lagged variables for each feature
for feature in features_to_lag:
    if feature in df.columns:
        # Create a lagged column for the feature
        lagged_column_name = f"lag_{feature}"
        df[lagged_column_name] = df.groupby('GEO_ID')[feature].shift(1)

        # For 2010, explicitly backfill lagged values with the same 2010 value
        df.loc[df['Year'] == 2010, lagged_column_name] = df.loc[df['Year'] == 2010, feature]

        # Debugging check: count remaining NaNs for 2010
        remaining_nans = df[df['Year'] == 2010][lagged_column_name].isnull().sum()
        if remaining_nans > 0:
            print(f"WARNING: {remaining_nans} missing values remain in {lagged_column_name} for 2010 after interpolation.")
        else:
            print(f"SUCCESS: All 2010 values in {lagged_column_name} have been successfully filled.")


# Save the updated DataFrame
df.to_csv(output_file_path, index=False)
print(f"Data with lagged variables saved to: {output_file_path}")


SUCCESS: All 2010 values in lag_P_3_BEDROOMS have been successfully filled.
SUCCESS: All 2010 values in lag_P_HH_INCOME_200K_PLUS have been successfully filled.
SUCCESS: All 2010 values in lag_P_SCHOOL_NURSERY have been successfully filled.
SUCCESS: All 2010 values in lag_AVG_COMMUTE_TIME have been successfully filled.
SUCCESS: All 2010 values in lag_P_GRAD_DEGREE have been successfully filled.
SUCCESS: All 2010 values in lag_P_1_BEDROOM have been successfully filled.
SUCCESS: All 2010 values in lag_P_1_UNIT_DETACHED have been successfully filled.
SUCCESS: All 2010 values in lag_P_COMMUTE_PUBLIC have been successfully filled.
SUCCESS: All 2010 values in lag_P_WORK_FINANCE have been successfully filled.
SUCCESS: All 2010 values in lag_P_WORK_INFORMATION have been successfully filled.
SUCCESS: All 2010 values in lag_P_HH_INCOME_50K_75K have been successfully filled.
SUCCESS: All 2010 values in lag_P_BACHELORS_PLUS have been successfully filled.
SUCCESS: All 2010 values in lag_P_LABOR_FOR